In [ ]:
import model
import tf_keras as keras
import pandas as pd
import tensorflow as tf
import crystal_loader
from tqdm import tqdm
import tqdm.keras

from symmetry import *
import dill
from sklearn.preprocessing import StandardScaler
import h5py
from sklearn.model_selection import train_test_split
print("hi")

In [ ]:
from sklearn.neighbors import KernelDensity

def inverse_density_weights(y, **kwargs):
    kde = KernelDensity(kernel='gaussian', bandwidth=5.0).fit(y.reshape(-1, 1))

    # 2. Estimate density
    log_density = kde.score_samples(y.reshape(-1, 1))
    density = np.exp(log_density)

    # 3. Invert density for weights
    weights = 1 / (density + 1e-6)
    weights /= np.mean(weights)  # optional normalization

    return weights.reshape(-1, 1)


In [ ]:
pickles_path = "../pickles/"
DNAME = "TiO2_2015_corrected"

features, labels = crystal_loader.load_dset(pickles_path, DNAME)

labels = np.array(labels)

def scale_ragged(features):
    stacked = np.vstack(features)
    SSC = StandardScaler().fit(stacked)
    scaled_features = [SSC.transform(struct) for struct in features]

    return scaled_features

scaled_features = scale_ragged(features)

Xtrain, Xtest, y_train, y_test = train_test_split(scaled_features, labels, shuffle=True, random_state=12, test_size=0.4)
Xval, Xtest, y_val, y_test = train_test_split(Xtest, y_test, shuffle=True, random_state=12, test_size=0.5)

Xtrain = tf.ragged.constant(Xtrain, ragged_rank=1, inner_shape=(70,))
Xval = tf.ragged.constant(Xval, ragged_rank=1, inner_shape=(70,))
Xtest = tf.ragged.constant(Xtest, ragged_rank=1, inner_shape=(70,))

y_train_weights = inverse_density_weights(y_train)
y_val_weights = inverse_density_weights(y_val)

In [ ]:
from importlib import reload
model = reload(model)
MLPNet = model.MLPNet
layers = [keras.layers.Dense(100, activation="relu"),
          keras.layers.Dropout(0.5),
          keras.layers.Dense(200, activation="relu"),
          keras.layers.Dropout(0.3),
          keras.layers.Dense(70, activation="relu")]
MLP1 = MLPNet(layers=layers, N_features=70, ragged_processing=False)
MLP1.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0003),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.RootMeanSquaredError()]
)

In [ ]:
print(MLP1.predict(Xtest))
print(Xtrain.shape[0])

In [ ]:
res = MLP1.fit(
    Xtrain, y_train, sample_weight=y_train_weights,
    batch_size = 20,
    epochs = 600,
    verbose = 0,
    validation_data=(Xval, y_val),
    validation_freq=1,
    callbacks=[]
)

In [ ]:
print(Xtrain.shape)
<_FlatMapDataset element_spec=(TensorSpec(shape=(None, 70), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>
<_TakeDataset element_spec=(TensorSpec(shape=(None, None, 70), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(res.epoch, res.history["loss"], label="training")
ax.plot(res.epoch, res.history["val_loss"], label="validation")

ax.legend()

In [ ]:
(<tf.Tensor: shape=(23, 70), dtype=float64, numpy=
array([[ 0.60915222, -0.54250154,  0.41247663, ..., -0.5382522 ,
        -0.58113234, -0.6209245 ],
       [ 0.58086981, -0.58559897,  0.36098034, ..., -0.55451661,
        -0.59884095, -0.63351055],
       [-0.45610275, -0.86305391, -0.51770812, ..., -0.72319429,
        -0.7152919 , -0.6765231 ],
       ...,
       [-0.13484626, -0.03686642, -0.10438722, ..., -0.65592439,
        -0.65784615, -0.64131981],
       [ 0.52685375,  0.88439498,  0.62293384, ..., -0.36293858,
        -0.31609761, -0.24929953],
       [ 0.49241531,  0.69841872,  0.5813253 , ..., -0.35998991,
        -0.31859196, -0.2801338 ]])>, array([[1.],
       [1.],
       [1.]]), np.int64(23), array([-19516.82259542]))